In [ ]:
import oracledb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import os

# Crear carpeta de resultados
CARPETA_RESULTADOS = "resultado_analisis"
os.makedirs(CARPETA_RESULTADOS, exist_ok=True)

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


USER = ""
PASSWORD = ""

In [16]:
# ============================================
# 1. CONEXIÓN A ORACLE DATABASE
# ============================================
def conectar_oracle():
    """Establece conexión con Oracle Database en OCI"""
    try:
        connection = oracledb.connect(
            user=USER,  # o tu usuario
            password=PASSWORD,
            dsn="malackathon2025v2_high",  # nombre del servicio del tnsnames.ora
            config_dir="WALLET/",
            wallet_location="WALLET/",
            wallet_password="Malakathon@2025"  # si configuraste una
        )
        return connection
    except Exception as e:
        print(f"✗ Error de conexión: {e}")
        return None

In [17]:
# ============================================
# 2. CARGAR DATOS
# ============================================
def cargar_datos(connection, query="SELECT * FROM SALUDMENTAL"):
    """Carga datos desde Oracle a DataFrame de pandas"""
    try:
        df = pd.read_sql(query, connection)
        print(f"✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
        return df
    except Exception as e:
        print(f"✗ Error al cargar datos: {e}")
        return None


In [18]:
def analisis_descriptivo(df):
    """Análisis estadístico elemental de las variables"""
    
    print("\n" + "="*80)
    print("1. ANÁLISIS DESCRIPTIVO INICIAL")
    print("="*80)
    
    # Información general
    print("\n--- INFORMACIÓN GENERAL ---")
    print(f"Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")
    print(f"Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Tipos de datos
    print("\n--- TIPOS DE DATOS ---")
    tipos_datos = df.dtypes.value_counts()
    print(tipos_datos)
    
    # Clasificación de variables
    numericas = df.select_dtypes(include=[np.number]).columns.tolist()
    categoricas = df.select_dtypes(include=['object']).columns.tolist()
    fechas = df.select_dtypes(include=['datetime64']).columns.tolist()
    
    # Detectar columnas de fecha en formato object
    for col in categoricas.copy():
        try:
            pd.to_datetime(df[col], errors='raise')
            fechas.append(col)
            categoricas.remove(col)
        except:
            pass
    
    print(f"\nVariables numéricas ({len(numericas)}): {numericas}")
    print(f"Variables categóricas ({len(categoricas)}): {categoricas}")
    print(f"Variables de fecha ({len(fechas)}): {fechas}")
    
    # Valores nulos
    print("\n--- VALORES NULOS Y DESCONOCIDOS ---")
    nulos = df.isnull().sum()
    porcentaje_nulos = (nulos / len(df) * 100).round(2)
    
    tabla_nulos = pd.DataFrame({
        'Variable': nulos.index,
        'Valores Nulos': nulos.values,
        '% Nulos': porcentaje_nulos.values
    })
    tabla_nulos = tabla_nulos[tabla_nulos['Valores Nulos'] > 0].sort_values('Valores Nulos', ascending=False)
    
    if len(tabla_nulos) > 0:
        print(tabla_nulos.to_string(index=False))
    else:
        print("✓ No hay valores nulos en el dataset")
    
    # Estadísticas descriptivas para variables numéricas
    if len(numericas) > 0:
        print("\n--- ESTADÍSTICAS DESCRIPTIVAS (Variables Numéricas) ---")
        print(df[numericas].describe().round(2))

        # === NUEVO BLOQUE AÑADIDO ===
        print("\n--- MEDIA, VARIANZA Y DESVIACIÓN TÍPICA ---")
        resumen_estadistico = pd.DataFrame({
            'Media': df[numericas].mean(),
            'Varianza': df[numericas].var(),
            'Desviación Típica': df[numericas].std()
        }).round(2)

    print(resumen_estadistico)
    
    # Guardar resultados dentro de la carpeta resultado_analisis
    resumen_path = os.path.join(CARPETA_RESULTADOS, 'resumen_estadistico.csv')
    resumen_estadistico.to_csv(resumen_path, index=True)
    print(f"✓ Resumen estadístico guardado: {resumen_path}")

    
    # Frecuencias para variables categóricas
    if len(categoricas) > 0:
        print("\n--- ANÁLISIS DE VARIABLES CATEGÓRICAS ---")
        for col in categoricas[:5]:  # Primeras 5 categóricas
            print(f"\n{col}:")
            print(f"  Valores únicos: {df[col].nunique()}")
            print(f"  Top 5 valores más frecuentes:")
            print(df[col].value_counts().head().to_string())
    
    return {
        'numericas': numericas,
        'categoricas': categoricas,
        'fechas': fechas,
        'nulos': tabla_nulos
    }

In [19]:
def detectar_outliers(df, columnas_numericas):
    """Detecta outliers usando método IQR y Z-score"""
    
    print("\n--- DETECCIÓN DE OUTLIERS ---")
    
    outliers_info = {}
    
    for col in columnas_numericas:
        # Método IQR
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        
        outliers_iqr = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]
        
        # Método Z-score (corregido)
        data_sin_nulos = df[col].dropna()
        if len(data_sin_nulos) > 0:
            z_scores = np.abs(stats.zscore(data_sin_nulos))
            outliers_z_mask = z_scores > 3
            num_outliers_z = np.sum(outliers_z_mask)
        else:
            num_outliers_z = 0
        
        if len(outliers_iqr) > 0:
            outliers_info[col] = {
                'IQR': len(outliers_iqr),
                'Z-score': num_outliers_z,
                'porcentaje': round(len(outliers_iqr) / len(df) * 100, 2)
            }
            print(f"\n{col}:")
            print(f"  Outliers (IQR): {len(outliers_iqr)} ({outliers_info[col]['porcentaje']}%)")
            print(f"  Outliers (Z-score > 3): {num_outliers_z}")
            print(f"  Rango normal: [{limite_inferior:.2f}, {limite_superior:.2f}]")
    
    if not outliers_info:
        print("✓ No se detectaron outliers significativos")
    
    return outliers_info


In [20]:
def crear_visualizaciones(df, info_variables):
    """Crea visualizaciones para el análisis exploratorio"""
    
    numericas = info_variables['numericas']
    categoricas = info_variables['categoricas']
    
    # 1. Distribución de variables numéricas
    if len(numericas) > 0:
        n_cols = min(3, len(numericas))
        n_rows = (len(numericas) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if n_rows > 1 else [axes]
        
        for idx, col in enumerate(numericas):
            if idx < len(axes):
                df[col].hist(bins=30, ax=axes[idx], edgecolor='black')
                axes[idx].set_title(f'Distribución de {col}')
                axes[idx].set_xlabel(col)
                axes[idx].set_ylabel('Frecuencia')
        
        # Ocultar ejes vacíos
        for idx in range(len(numericas), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(CARPETA_RESULTADOS, 'distribucion_numericas.png'), dpi=300, bbox_inches='tight')
        print("\n✓ Gráfico guardado: distribucion_numericas.png")
        plt.close()
    
    # 2. Boxplots para detección de outliers
    if len(numericas) > 0:
        n_cols = min(3, len(numericas))
        n_rows = (len(numericas) + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if n_rows > 1 else [axes]
        
        for idx, col in enumerate(numericas):
            if idx < len(axes):
                df.boxplot(column=col, ax=axes[idx])
                axes[idx].set_title(f'Boxplot de {col}')
                axes[idx].set_ylabel(col)
        
        for idx in range(len(numericas), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(CARPETA_RESULTADOS, 'boxplots_outliers.png'), dpi=300, bbox_inches='tight')
        print("✓ Gráfico guardado: boxplots_outliers.png")
        plt.close()
    
    # 3. Matriz de correlación
    if len(numericas) > 1:
        plt.figure(figsize=(12, 10))
        correlation_matrix = df[numericas].corr()
        sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                    center=0, square=True, linewidths=1)
        plt.title('Matriz de Correlación', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(os.path.join(CARPETA_RESULTADOS, 'matriz_correlacion.png'), dpi=300, bbox_inches='tight')
        print("✓ Gráfico guardado: matriz_correlacion.png")
        plt.close()
    
    # 4. Distribución de variables categóricas
    if len(categoricas) > 0:
        for col in categoricas[:5]:  # Primeras 5 categóricas
            plt.figure(figsize=(12, 6))
            df[col].value_counts().head(10).plot(kind='bar', edgecolor='black')
            plt.title(f'Distribución de {col}', fontsize=14, fontweight='bold')
            plt.xlabel(col)
            plt.ylabel('Frecuencia')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(os.path.join(CARPETA_RESULTADOS, f'distribucion_{col}.png'), dpi=300, bbox_inches='tight')
            print(f"✓ Gráfico guardado: distribucion_{col}.png")
            plt.close()

In [21]:
def ingenieria_caracteristicas(df, info_variables):
    """
    Crea nuevas variables útiles de forma controlada y coherente,
    evitando generar variables redundantes o poco informativas.
    """

    print("\n" + "="*80)
    print("2. INGENIERÍA DE CARACTERÍSTICAS")
    print("="*80)
    
    df_nuevo = df.copy()
    nuevas_variables = []

    # A. Transformación de variables de fecha
    fechas = info_variables.get('fechas', [])
    for col in fechas:
        if col not in df_nuevo.columns:
            print(f"⚠ Columna {col} no encontrada, se omite.")
            continue

        try:
            if not np.issubdtype(df_nuevo[col].dtype, np.datetime64):
                df_nuevo[col] = pd.to_datetime(df_nuevo[col], errors='coerce')
            
            if df_nuevo[col].isna().all():
                print(f"⚠ {col} no contiene fechas válidas, se omite.")
                continue
            
            # Extraer componentes útiles
            for comp, func in zip(
                ['año', 'mes', 'dia', 'dia_semana', 'trimestre'],
                [lambda x: x.dt.year, lambda x: x.dt.month, lambda x: x.dt.day,
                 lambda x: x.dt.dayofweek, lambda x: x.dt.quarter]
            ):
                nueva = f"{col}_{comp}"
                df_nuevo[nueva] = func(df_nuevo[col])
                nuevas_variables.append(nueva)

            print(f"✓ Características extraídas de {col}: Año, Mes, Día, Día de la semana, Trimestre")

        except Exception as e:
            print(f"⚠ No se pudieron extraer características de {col}: {e}")

    # B. Binning de variables numéricas
    numericas = info_variables.get('numericas', [])
    for col in numericas[:3]:
        if col not in df_nuevo.columns:
            continue

        valores_unicos = df_nuevo[col].nunique()
        if valores_unicos < 5:
            print(f"⚠ {col} tiene pocos valores distintos ({valores_unicos}), no se binea.")
            continue

        try:
            df_nuevo[f'{col}_bin'] = pd.qcut(
                df_nuevo[col],
                q=4,
                labels=['Bajo', 'Medio-Bajo', 'Medio-Alto', 'Alto'],
                duplicates='drop'
            )
            nuevas_variables.append(f'{col}_bin')
            print(f"✓ Variable binned creada: {col}_bin (cuartiles)")
        except Exception:
            print(f"⚠ No se pudo crear bins para {col} (valores fuera de rango o constantes)")

    # C. Codificación de variables categóricas
    categoricas = info_variables.get('categoricas', [])
    print("\n--- CODIFICACIÓN DE VARIABLES CATEGÓRICAS ---")

    for col in categoricas:
        if col not in df_nuevo.columns:
            continue

        n_unique = df_nuevo[col].nunique(dropna=True)
        if n_unique == 1:
            print(f"⚠ {col} tiene una sola categoría, se omite.")
            continue

        if n_unique <= 10:
            dummies = pd.get_dummies(df_nuevo[col], prefix=col, drop_first=True)
            if dummies.shape[1] > 0:
                df_nuevo = pd.concat([df_nuevo, dummies], axis=1)
                nuevas_variables.extend(dummies.columns.tolist())
                print(f"✓ One-Hot Encoding aplicado a {col} ({n_unique} categorías)")
        else:
            df_nuevo[f'{col}_encoded'] = pd.Categorical(df_nuevo[col]).codes
            nuevas_variables.append(f'{col}_encoded')
            print(f"✓ Label Encoding aplicado a {col} ({n_unique} categorías)")

    # D. Interacciones entre variables numéricas
    if len(numericas) >= 2:
        print("\n--- CREACIÓN DE VARIABLES DE INTERACCIÓN ---")
        col1, col2 = numericas[0], numericas[1]

        if col1 in df_nuevo.columns and col2 in df_nuevo.columns:
            # Solo crear si hay variabilidad
            if df_nuevo[col1].std() > 0 and df_nuevo[col2].std() > 0:
                df_nuevo[f'{col1}_x_{col2}'] = df_nuevo[col1] * df_nuevo[col2]
                nuevas_variables.append(f'{col1}_x_{col2}')
                print(f"✓ Variable de interacción creada: {col1}_x_{col2}")

                df_nuevo[f'{col1}_div_{col2}'] = df_nuevo[col1] / (df_nuevo[col2] + 1e-6)
                nuevas_variables.append(f'{col1}_div_{col2}')
                print(f"✓ Variable ratio creada: {col1}_div_{col2}")
            else:
                print(f"⚠ {col1} o {col2} tienen desviación cero, no se crean interacciones.")

    # E. Variables de agregación
    if categoricas and numericas:
        print("\n--- VARIABLES DE AGREGACIÓN ---")
        cat_col = categoricas[0]
        num_col = numericas[0]

        if cat_col in df_nuevo.columns and num_col in df_nuevo.columns:
            try:
                df_nuevo[f'{num_col}_mean_by_{cat_col}'] = df_nuevo.groupby(cat_col)[num_col].transform('mean')
                nuevas_variables.append(f'{num_col}_mean_by_{cat_col}')
                print(f"✓ Variable agregada: {num_col}_mean_by_{cat_col}")
            except Exception as e:
                print(f"⚠ No se pudo crear variable agregada: {e}")

    print(f"\n{'='*80}")
    print(f"RESUMEN: Se crearon {len(nuevas_variables)} nuevas características")
    print(f"Dataset original: {df.shape[1]} columnas")
    print(f"Dataset transformado: {df_nuevo.shape[1]} columnas")
    print(f"{'='*80}")

    return df_nuevo, nuevas_variables


In [22]:
def exportar_resultados(df_original, df_transformado, info_variables, nuevas_vars):
    """Exporta resultados a CSV para generar el PDF"""
    
    # Guardar dataset transformado
    df_transformado.to_csv(os.path.join(CARPETA_RESULTADOS, 'dataset_transformado.csv'), index=False)


    print("\n✓ Dataset transformado guardado: dataset_transformado.csv")
    
    # Guardar resumen de análisis
    with open(os.path.join(CARPETA_RESULTADOS, 'resumen_analisis.txt'), 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("RESUMEN DEL ANÁLISIS EXPLORATORIO DE DATOS\n")
        f.write("="*80 + "\n\n")
        
        f.write(f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("1. INFORMACIÓN GENERAL\n")
        f.write(f"   - Filas: {df_original.shape[0]}\n")
        f.write(f"   - Columnas originales: {df_original.shape[1]}\n")
        f.write(f"   - Columnas después de transformación: {df_transformado.shape[1]}\n\n")
        
        f.write("2. TIPOS DE VARIABLES\n")
        f.write(f"   - Variables numéricas: {len(info_variables['numericas'])}\n")
        f.write(f"   - Variables categóricas: {len(info_variables['categoricas'])}\n")
        f.write(f"   - Variables de fecha: {len(info_variables['fechas'])}\n\n")
        
        f.write("3. NUEVAS CARACTERÍSTICAS CREADAS\n")
        f.write(f"   - Total: {len(nuevas_vars)}\n")
        for var in nuevas_vars[:20]:  # Primeras 20
            f.write(f"   - {var}\n")
        if len(nuevas_vars) > 20:
            f.write(f"   ... y {len(nuevas_vars) - 20} más\n")
    
    print("✓ Resumen guardado: resumen_analisis.txt")
                                                                                                                                                                                                    

In [23]:
def ejecutar_analisis():
    """Función principal que ejecuta todo el análisis"""
    
    print("="*80)
    print("ANÁLISIS EXPLORATORIO DE DATOS - ORACLE DATABASE")
    print("="*80)
    
    # Conectar a Oracle
    conn = conectar_oracle()
    if conn is None:
        return
    query = "SELECT * FROM VISTA_IA"
    # Cargar datos (personaliza tu query)
    df = cargar_datos(conn, query)
    
    if df is None:
        conn.close()
        return
    
    # Cerrar conexión
    conn.close()
    
    # Análisis descriptivo
    info_vars = analisis_descriptivo(df)
    
    # Detección de outliers
    if len(info_vars['numericas']) > 0:
        detectar_outliers(df, info_vars['numericas'])
    
    # Crear visualizaciones
    crear_visualizaciones(df, info_vars)
    
    # Ingeniería de características
    df_transformado, nuevas_vars = ingenieria_caracteristicas(df, info_vars)
    
    # Exportar resultados
    exportar_resultados(df, df_transformado, info_vars, nuevas_vars)
    
    print("\n" + "="*80)
    print("✓ ANÁLISIS COMPLETADO")
    print("="*80)
    print("\nArchivos generados:")
    print("  - distribucion_numericas.png")
    print("  - boxplots_outliers.png")
    print("  - matriz_correlacion.png")
    print("  - distribucion_[variable].png (por cada categórica)")
    print("  - dataset_transformado.csv")
    print("  - resumen_analisis.txt")
    print("  - resumen_estadistico.csv")

In [24]:
ejecutar_analisis()

ANÁLISIS EXPLORATORIO DE DATOS - ORACLE DATABASE
✓ Datos cargados: 21210 filas, 13 columnas

1. ANÁLISIS DESCRIPTIVO INICIAL

--- INFORMACIÓN GENERAL ---
Dimensiones: 21210 filas × 13 columnas
Memoria utilizada: 6.71 MB

--- TIPOS DE DATOS ---
int64     9
object    4
Name: count, dtype: int64

Variables numéricas (9): ['SEXO', 'ESTANCIA_DÍAS', 'NIVEL_SEVERIDAD_APR', 'RIESGO_MORTALIDAD_APR', 'EDAD', 'CONTINUIDAD_ASISTENCIAL', 'INGRESO_EN_UCI', 'EDAD_EN_INGRESO', 'COSTE_APR']
Variables categóricas (4): ['Diagnóstico Principal', 'Diagnóstico 2', 'Diagnóstico 3', 'COMUNIDAD_AUTÓNOMA']
Variables de fecha (0): []

--- VALORES NULOS Y DESCONOCIDOS ---
     Variable  Valores Nulos  % Nulos
Diagnóstico 3           6150    29.00
Diagnóstico 2           2604    12.28

--- ESTADÍSTICAS DESCRIPTIVAS (Variables Numéricas) ---
           SEXO  ESTANCIA_DÍAS  NIVEL_SEVERIDAD_APR  RIESGO_MORTALIDAD_APR  \
count  21210.00       21210.00             21210.00               21210.00   
mean       1.45     